# Install Required Packages

In [1]:
!pip install langchain
!pip install openai
!pip install tiktoken
!pip install faiss-gpu
!pip install langchain_experimental
!pip install "langchain[docarray]"
!pip install -U langchain-openai
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.3/377.3 kB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

# Config : Keys

In [2]:
import os

# Prompt the user for their OpenAI API key
api_key = "ENTER YOUR API KEY HERE"

# Set the API key as an environment variable
os.environ["OPENAI_API_KEY"] = api_key

# Optionally, check that the environment variable was set correctly
print("OPENAI_API_KEY has been set!")

OPENAI_API_KEY has been set!


In [3]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain_openai import ChatOpenAI

# Loading Data

In [4]:
pdf_file_path = '/content/handbook.pdf'
loader = PyPDFLoader(pdf_file_path)
pages = loader.load_and_split()

In [5]:
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(pages, embedding=embeddings)

In [25]:
llm = ChatOpenAI(temperature=0.8, model_name="gpt-4o-mini")

In [26]:
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

In [27]:
conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
    memory=memory
)

# Inference

In [12]:
query = "Who is the CEO of the company?"

result = conversation_chain({"question": query})
answer = result["answer"]

output = {"question":query,
          "answer":answer}

print(output)

{'question': 'Who is the CEO of the company?', 'answer': 'The CEO of the company is Shruti Gupta.'}


In [14]:
def call_llm(query):
    # Assuming conversation_chain is defined elsewhere and returns the appropriate result
    result = conversation_chain({"question": query})
    answer = result["answer"]

    output = {
        "question": query,
        "answer": answer
    }

    return output

# Example usage
query = "Who is the CEO of the company?"
call_llm(query)


{'question': 'Who is the CEO of the company?',
 'answer': 'The CEO of the company is Shruti Gupta.'}

# Push output to Slack

In [ ]:
import requests
import json

# Webhook URL (replace with your actual webhook URL)
webhook_url = 'ENTER YOUR WEBHOOK URL HERE'


# Example usage
query = input("Enter your query :-")

output_json = call_llm(query)
question = output_json["question"]
answer = output_json["answer"]



# Message payload
payload = {
    'text': 'Here is a new query and response:',
    'blocks': [
        {
            'type': 'section',
            'text': {
                'type': 'mrkdwn',
                'text': question
            }
        },
        {
            'type': 'section',
            'text': {
                'type': 'mrkdwn',
                'text': answer
            }
        }
    ]
}

# Send the message to Slack
response = requests.post(webhook_url, data=json.dumps(payload), headers={'Content-Type': 'application/json'})

# Check if the request was successful
if response.status_code == 200:
    print('Message posted successfully')
else:
    print(f'Failed to post message: {response.status_code}, {response.text}')


Enter your query :-What is the termination policy?
Message posted successfully
